In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import cvxpy as cp

In [ ]:
#Parameters

Pmax = 5
n_c = 0.95
n_d = 0.95
Cmax = 10
C_0 = 70
C_n = 80

# Define the prices array
l = np.array([0.5, 1.4, 4.4, 9.4, 6.4, 2.3]) #these price arrays should be DK2
print(l)

In [ ]:
#Optimization Variables

n = 6 # Number of steps
p_c = cp.Variable(n)
p_d = cp.Variable(n)
X   = cp.Variable(n)

In [ ]:
#objective function

cost = cp.sum(-p_d@l + p_c@l)

In [ ]:
#Constraints

constraints = [p_c >= 0,
               p_d >= 0,
               p_c <= Pmax,
               p_d <= Pmax]
constraints += [X >= 0, X <= Cmax]
constraints += [X[0]==C_0 + p_c[0]*n_c - p_d[0]/n_d]

for j in range(1,n):
    constraints += [X[j]==X[j-1] + p_c[j]*n_c - p_d[j]/n_d]

constraints += [X[n-1]>=C_n]

In [ ]:
#Problem 

problem = cp.Problem(cp.Minimize(cost), constraints)
problem.solve(solver=cp.CLARABEL)

print("\n--- Optimization Results ---")
print(f"Charging power (p_c): {np.round(p_c.value, 1)}")
print(f"Discharging power (p_d): {np.round(p_d.value, 1)}")
print(f"State of charge (X): {np.round(X.value, 1)}")
print(f"Total cost: {np.round(cost.value, 1)}")
print("-----------------------------")

In [ ]:
#Results

# Create the figure and axes objects for the two subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 6))
fig.subplots_adjust(hspace=0.4)  # Adjust space between plots

# Plot the prices in the top subplot (exclude the first and last hours)
ax1.stairs(l, range(len(l) + 1), label='Prices', baseline = None, color='darkblue', linewidth=2)
ax1.set_xlabel('Hour', fontsize=12)
ax1.set_ylabel('Price [DKK/kWh]', fontsize=12)
ax1.set_title("Spot Prices Over Time", fontsize=14, fontweight='bold')
ax1.grid(True, linestyle=':', linewidth=0.7, alpha=0.8)

# Plot the power in the bottom subplot (exclude the first and last hours)
ax2.stairs(p_c.value, range(len(l) + 1), label='Charging Power', baseline = None, color='green', linewidth=2)
ax2.stairs(-p_d.value, range(len(l) + 1), label='Discharging Power', baseline = None, color='red', linewidth=2)
ax2.set_xlabel('Hour', fontsize=12)
ax2.set_ylabel('Power [kW]', fontsize=12)
ax2.set_title("Battery Charging/Discharging Schedule", fontsize=14, fontweight='bold')
ax2.legend(loc='upper center', fontsize=10, frameon=True, shadow=True, ncol=2)
ax2.grid(True, linestyle=':', linewidth=0.7, alpha=0.8)

# Show the plot
plt.tight_layout()
plt.show()